# Fouille de données et medias sociaux
# TP4 : Bagging et Boosting

## Exercice 1 Bagging
### Question 1
Écrivez une fonction (ou une classe) implémentant la méthode Bagging.

In [1]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.metrics import accuracy_score

# TODO: Parallelization
# TODO: Bootstrap on features

class BaggingClassifier(ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=5, 
                 max_samples=1.0, bootstrap=True, verbose=0,
                oob_score=False):
        if base_estimator is None:
            self.base_estimator = DecisionTreeClassifier
        else:
            self.base_estimator = base_estimator
            
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.bootstrap = bootstrap
        self.verbose = verbose
        self.oob_score = oob_score
        
        
        self.estimators = None
        self.oob_score_ = None
        
    def fit(self, X, y):
        self.estimators = []
        if self.oob_score:
            self.oob_counters = [Counter() for elt in X]
            # self.oob_counters[i] = predictions for sample i
        if self.verbose >= 1:
            print("##### Fitting the classifiers #####")
        for i in range(self.n_estimators):
            if self.verbose >= 1:
                print("Fitting classifier n°%d..." % i)
            clf = self.base_estimator()
            sample_idx = np.random.choice(X.shape[0], 
                                          int(X.shape[0]*self.max_samples), 
                                          replace=self.bootstrap)
            if self.verbose >= 2:
                print("Random indexes:", sample_idx)
            clf.fit(X[sample_idx], y[sample_idx])
            if self.verbose >= 1:
                print("Done.")
            
            if self.oob_score:
                oob_samples = np.setdiff1d(np.arange(X.shape[0]), sample_idx)
                preds = clf.predict(X)
                for i in oob_samples:
                    self.oob_counters[i].update([preds[i]])
                    
            self.estimators.append(clf)
          
        if self.oob_score:
            if Counter() in self.oob_counters:
                print("Warning, some samples don't have OOB estimations, \
                        the number of estimators seems to be too low.")
            else:
                self.oob_predictions = [c.most_common(1)[0][0] for c in self.oob_counters]
                errors = np.count_nonzero(self.oob_predictions != y)
                self.oob_score_ = errors / X.shape[0]
        
        if self.verbose >= 1:
            print("##### Fitting is over #####")
        
    def predict(self, X):
        predictions = np.empty((X.shape[0], len(self.estimators)))
        # predictions[i][j] = self.estimators[j].predict(X[i])
        # TODO: More efficient method: only store new class and counters
        for j, clf in enumerate(self.estimators):
            predictions[:, j] = clf.predict(X)
        
        most_common = [Counter(line).most_common(1)[0][0] for line in predictions]
        return most_common
        
    def score(self, X, y):
        return accuracy_score(self.predict(X), y)
    
    
    

### Question 2
Appliquez cette méthode sur des arbres de décisions.

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV

data = datasets.load_iris()
X, y = data.data, data.target

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Classes in y:", np.unique(y, return_counts=True))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

bag_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier, oob_score=True,
                            n_estimators=100, verbose=0)
bag_clf.fit(X_train, y_train)
print("Scores with bag_clf:", bag_clf.score(X_test, y_test))



Shape of X: (150, 4)
Shape of y: (150,)
Classes in y: (array([0, 1, 2]), array([50, 50, 50]))
Scores with bag_clf: 0.955555555556


### Question 3
Évaluez et commentez l’erreur en généralisation par rapport à un arbre unique.

In [3]:
print("For the BaggingClassifier, OOB error:", bag_clf.oob_score_)

# Evaluating the OOB error on a decision tree:

# Parameters for the Grid Search:
params = {"criterion":("gini", "entropy"),
          "splitter":("best", "random"),
          "max_depth":np.arange(5, 51, 5),
          "min_samples_split":np.linspace(1e-3, 1., 5)
            }

gridsearch = GridSearchCV(DecisionTreeClassifier(), params)
gridsearch.fit(X_train, y_train)

print("For the DecisionTreeClassifier, OOB error:", 1 - gridsearch.score(X_test, y_test))
print("Best parameters found for the DecisionTreeClassifier:", gridsearch.best_params_)

For the BaggingClassifier, OOB error: 0.06666666666666667
For the DecisionTreeClassifier, OOB error: 0.0888888888889
Best parameters found for the DecisionTreeClassifier: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 0.25074999999999997, 'splitter': 'random'}


## Exercice 2 Boosting
### Question 1
**Montrer que le coefficient de pondération des hypothèses vaut bien ce qu’il vaut.**

Soit un problème d'apprentissage avec $((x_i, y_i)_{i \in \{1..n\}}$, $x_i \in X$, $y_i \in \{-1, +1\}$

Après l'itération n° $m-1$, notre classifieur Boost est une combinaison linéaire de classifieurs faibles $h_i:X \rightarrow \{-1, +1\}$ pondéré par les coefficients $a_i$:

$C_{m-1}(x) = \sum_{i=1}^{m-1}{a_i ⋅ h_i(x)}$.

On veut trouver un nouvel estimateur faible $h_m$ pour avoir l'estimateur Boost à l'itération $m$.

Soit l'erreur totale de $C_m$ : $E = \sum_{i=1}^n{e^{-y_i ⋅ C_m(x_i)}}$, 

posons $w_i^1 = 1$ et $w_i^m = e^{-y_i ⋅ C_{m-1}(x_i)}$ pour $1 < m$, 
on a alors :

$E = \sum_{i=1}^n{e^{-y_i ⋅ C_m(x_i)}} \\
E = \sum_{y_i = h_m(x_i)}{w_i^m⋅e^{-a_m}} + \sum_{y_i \neq h_m(x_i)}{w_i^m⋅e^{a_m}} \\
E = \sum_{i=1}^n{w_i^m ⋅ e^{-a_m}} + \sum_{y_i \neq h_m(x_i)}{w_i^m⋅(e^{a_m} - e^{-a_m})} $

Ainsi l'estimateur faible $h_m$ qui minimise $E$ est celui qui minimise $\sum_{y_i \neq h_m(x_i)}{w_i^m}$, c'est-à-dire le classifieur qui fait le moins d'erreurs de classification, avec la pondération $w_i^m$.

Après avoir trouvé ce classifieur $h_m$, il faut trouver le coefficient $a_m$, c'est le coefficient qui minimise $E$ :

$\frac {dE}{da_m} = \sum_{y_i \neq h_m(x_i)}{w_i^m⋅e^{a_m}} - \sum_{y_i = h_m(x_i)}{w_i^m⋅e^{-a_m}}$

$\frac {dE}{da_m} = 0 \Leftrightarrow \sum_{y_i \neq h_m(x_i)}{w_i^m⋅e^{a_m}} - \sum_{y_i = h_m(x_i)}{w_i^m⋅e^{-a_m}} = 0 \\
\Leftrightarrow \sum_{y_i \neq h_m(x_i)}{w_i^m⋅e^{a_m}}  = \sum_{y_i = h_m(x_i)}{w_i^m⋅e^{-a_m}}\\
\Leftrightarrow e^{2⋅a_m}⋅\sum_{y_i \neq h_m(x_i)}{w_i^m}  = \sum_{y_i = h_m(x_i)}{w_i^m}\\
\Leftrightarrow 2⋅a_m = ln(\frac {\sum_{y_i = h_m(x_i)}{w_i^m})}{\sum_{y_i \neq h_m(x_i)}{w_i^m} })\\
\Leftrightarrow a_m = \frac 1 2 ln(\frac {\sum_{y_i = h_m(x_i)}{w_i^m})}{\sum_{y_i \neq h_m(x_i)}{w_i^m} })$


### Question 2
Écrivez une fonction (ou une classe) implémentant la méthode AdaBoost.

### Question 3
Comment adapter un classifieur qui ne gère pas naturellement des pondérations pour qu’il puisse en tenir
compte ?
### Question 4
Appliquez cette méthode sur des stumps (arbres de décision à un nœud).
### Question 5
Tracez les courbes de l’erreur d’apprentissage et de l’erreur de test en fonction du nombre de classifieurs.
1
### Question 6
Affichez les points associés aux poids les plus élevés (donc les plus difficiles à classer). Sont-ils bien classés
désormais ? Étaient-ils bien classé avec un seul arbre de décision ? Avec le Bagging ?
### Question 7
Appliquez AdaBoost à des arbres de décision de profondeur plus grande.
Comment se comporte la généralisation quand la profondeur des arbres augmente ?
2